In [1]:
import numpy
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
from importlib import reload
import InputList
import BinaryReader
import Preprocessor
import models
import Callbacks
import InputListUtils

2022-02-21 10:11:39.045030: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-02-21 10:11:41.237493: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-02-21 10:11:41.237615: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-02-21 10:11:41.238744: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-21 10:11:41.942382: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1365] Profiler found 2 GPUs
2022-02-21 10:11:41.943225: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcupti.so.10.1
2022-02-21 10:11:41.944450: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-02-21 10:11:41.944606: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1487] CUPTI activity buffer flushed

In [7]:
reload(InputListUtils)

<module 'InputListUtils' from '/home/juliusmlcaesar/PycharmProjects/oct-classifier/InputListUtils.py'>

In [2]:
InputListUtils.generate_diabetic_files_list()
tr_files = InputList.diabetic_training_files + InputList.healthy_training_files
len(tr_files)

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
reader = BinaryReader.BinaryReader()  # TODO: Normalizer
training_dataset, validation_dataset = reader.create_training_datasets(tr_files)
preprocesser = Preprocessor.Preprocessor(training_dataset)

2022-02-17 10:23:07.593822: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-17 10:23:07.596532: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-02-17 10:23:07.598900: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:21:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-02-17 10:23:07.598952: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-02-17 10:23:07.601131: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-02-17 10:23:07.6011

Num GPUs Available:  2
[('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D95/rechts/raw_1536x2048x2045x2_5217.bin', 1), ('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D60 FU1/links/raw_1536x2048x2045x2_7051.bin', 1), ('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D73/links/raw_1536x2048x2045x2_29875.bin', 1), ('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D62/links/raw_1536x2048x2045x2_7930.bin', 1), ('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D105/links/raw_1536x2048x2045x2_29122.bin', 1), ('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D77/links/raw_1536x2048x2045x2_3140.bin', 1), ('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D92/links/raw_1536x2048x2045x2_21336.bin', 1), ('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D63/links/raw_1536x2048x2045x2_29802.bin', 1), ('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D77/rechts/raw_1536x2048x2045x2_2637.bin', 1), ('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D48/rechts/raw_1536x2

2022-02-17 10:23:08.223247: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-17 10:23:08.553372: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-02-17 10:23:08.554816: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:21:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-02-17 10:23:08.554863: I tensorflow/stream_executor/platform/default/dso_loader.cc:49

In [6]:
a = len(InputList.diabetic_training_files)
b = len(InputList.healthy_training_files)
class_weights = {0: (1/b) * (a+b)/2, 1:(1/a) * (a+b)/2}
class_weights

{0: 13.666666666666666, 1: 0.5189873417721519}

In [7]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    print("Creating Normalization Layer:")
    normalization_layer = preprocesser.normalize_layer()

    model = models.RawClassifier().model()(training_dataset.element_spec[0].shape, normalization_layer, reconstruction=False)

    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF

    history = model.fit(
        preprocesser.batch(3),
        epochs=10,
        validation_data=Preprocessor.Preprocessor(validation_dataset).batch(3),
        callbacks=Callbacks.my_callbacks,
        class_weight=class_weights,
        use_multiprocessing=True
    )
    model.save('savedModels/first')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2
Creating Normalization Layer:


2022-02-17 10:23:20.914343: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-17 10:23:20.933614: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2399700000 Hz


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1536, 102, 102, 1 0         
_________________________________________________________________
normalization (Normalization (None, 1536, 102, 102, 1) 3         
_________________________________________________________________
conv3d (Conv3D)              

2022-02-17 10:26:12.749911: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "ParallelInterleaveDatasetV4/_7"
op: "ParallelInterleaveDatasetV4"
input: "TensorSliceDataset/_1"
input: "Const/_2"
input: "Const/_3"
input: "Const/_10"
input: "Const/_5"
input: "Const/_6"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "deterministic"
  value {
    s: "false"
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_interleave_lambda_43"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 1536
        }
        dim {
          size: 102
        }
        dim {
          size: 102
        }
        dim {
          size: 1
        }
      }
      shape {
 

Epoch 1/10


...

2022-02-17 10:26:16.533228: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-02-17 10:27:05.887129: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


     20/Unknown - 202s 3s/step - loss: 9.9611 

2022-02-17 10:29:35.310947: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-02-17 10:29:35.397791: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-02-17 10:29:35.431351: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-02-17 10:29:35.479671: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20220217-102255/train/plugins/profile/2022_02_17_10_29_35
2022-02-17 10:29:35.512759: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20220217-102255/train/plugins/profile/2022_02_17_10_29_35/hyperion-vie.trace.json.gz
2022-02-17 10:29:35.662034: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20220217-102255/train/plugins/profile/2022_02_17_10_29_35
2022-02-17 10:29:35.682503: I tensorflow/core

  17437/Unknown - 82777s 5s/step - loss: 12.7600

KeyboardInterrupt: 

In [ ]:
expected = 26133
expected_time = 26133 * 4 # sekunden -> 30 Stunden pro Epoche :(

In [7]:
#normalization_weights = normalization_layer.get_weights()
#print("Before:" + str(normalization_weights))
# normalization_weights_path = "results/normalization.npy"
# #np.save(normalization_weights_path, normalization_weights)
# normalization_layer.set_weights(np.load(normalization_weights_path))
# print("After" + str(normalization_layer.get_weights()))

In [8]:
model.save('savedModels/first_with_all_data')

2022-02-18 09:26:48.837202: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Assets written to: savedModels/first_with_all_data/assets
